In [1]:
from pymongo import MongoClient
import pandas as pd
import time
import re
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from dotenv import load_dotenv
import os
import requests
import warnings
warnings.filterwarnings("ignore")
from pymongo import GEOSPHERE
import json
#from cartoframes.viz import Map, Layer, popup_element
import geopandas as gpd
import geopy.distance


In [176]:
nyork_df = pd.read_csv("./dataset/new_york_data.csv")


In [3]:
load_dotenv()

True

In [4]:
token_fsq = os.getenv("token")

Now we have a data set with locations in NY, we are going to take the top 3 based on the density, and show them on a map with ametnities around.  
    we're also choosing the top 3 locations that are next to each other, because we want our company to be around other companies that had raised more thn 10M$.( so far we have been trying to choose a location of a company who had raised 10M\$, so we need to check the surroundings as well)



In [182]:
map2 = Map(location = ["40.751892", "-73.987592"], zoom_start = 15)
#Optimum Energy
folium.Circle([40.751892, -73.987592],radius=1000).add_to(map2)
#Crispy Gamer
folium.Circle([40.750597, -73.985273],radius=1000).add_to(map2)
#Heavy
folium.Circle([40.753385, -73.989620],radius=1000).add_to(map2)

In [183]:
map2

In [140]:
#add crispy game location

In [184]:
icon = Icon(color = "blue",
            opacity = 0.1,
            prefix = "fa",
            icon = "building",
            icon_color = "black"
)
Crispy_Gamer = Marker(
    location = [40.750597, -73.985273],
    tooltip = "Crispy Gamer",
    icon=icon,
    radius = 1000
)
Crispy_Gamer.add_to(map2)

In [185]:
#add "heavy" location

In [143]:
icon = Icon(color = "blue",
            opacity = 0.1,
            prefix = "fa",
            icon = "building",
            icon_color = "black"
)
Heavy = Marker(
    location = [40.753385, -73.989620],
    tooltip = "Heavy",
    icon=icon,
    radius = 1000
)
Heavy.add_to(map2)

In [186]:
#add Optimum_Energy location

In [145]:
icon = Icon(color = "blue",
            opacity = 0.1,
            prefix = "fa",
            icon = "building",
            icon_color = "black",
)
Optimum_Energy = Marker(
    location = [40.751892, -73.987592],
    tooltip = "Optimum Energy",
    icon=icon
)
Optimum_Energy.add_to(map2)

In [146]:
def getNearbyStarbucks(lat, long, radius=500):

    # create the API request URL
    url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&radius={radius}&chains=ab4c54c0-d68a-012e-5619-003048cad9da&exclude_all_chains=false&sort=DISTANCE&limit=10"        
    # make the GET request
    headers = {
    "accept": "application/json",
    "Authorization": token_fsq
    }
    
    response = requests.get(url,headers=headers).json()    
    return response
     

In [147]:
response_star = getNearbyStarbucks("40.751892","-73.987592")

In [148]:
starbucks = []
for i in response_star["results"]:
    
    name = i["name"]
    latitude = i["geocodes"]["main"]["latitude"]
    longitude = i["geocodes"]["main"]["longitude"]
    distance = i["distance"]
    
    starbucks.append({"name":name, "latitude":latitude, "longitude":longitude, "distance_Optimum Energy": distance})
    
    starbucks

In [149]:
df_cafe = pd.DataFrame(starbucks)


In [150]:
for index, row in df_cafe.iterrows():
    
    #1. MARKER without icon
    starbucks = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    
    #2. Icon
          
    icon = Icon (
            color="blue",
            opacity = 0.6,
            prefix = "fa",
            icon="coffee",
            icon_color = "black"
        )
     #3. Marker
    new_marker = Marker(**starbucks, icon = icon, radius = 100)
    
    #4. Add the Marker
    new_marker.add_to(map2)

In [151]:
new_marker.add_to(map2)

In [152]:
def getNearbyLocation(lat, long, categorie, radius=1000):

    # create the API request URL
    url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&radius={radius}&categories={categorie}&exclude_all_chains=false&sort=DISTANCE&limit=10"

    # make the GET request
    headers = {
    "accept": "application/json",
    "Authorization": token_fsq
    }    
    response = requests.get(url,headers=headers).json()    
    
    return response

In [153]:
night_life_response = getNearbyLocation("40.751892","-73.987592","13003")

In [154]:
night_life = []
for i in night_life_response["results"]:
    
    name = i["name"]
    latitude = i["geocodes"]["main"]["latitude"]
    longitude = i["geocodes"]["main"]["longitude"]
    distance = i["distance"]
    
    night_life.append({"name":name, "latitude":latitude, "longitude":longitude, "distance_Optimum Energy": distance})
    
    night_life

In [155]:
df_night_life = pd.DataFrame(night_life)
df_night_life

,name,latitude,longitude,distance_Optimum Energy
0,Bar Moxy,40.752206,-73.989137,144
1,Magic Hour Rooftop Bar & Lounge,40.752366,-73.989187,147
2,The Heist,40.751598,-73.985662,165
3,Monarch Rooftop,40.750485,-73.986929,179
4,The Keg Room,40.750887,-73.986028,183
5,Refinery Rooftop,40.752257,-73.985251,192
6,Stout,40.753321,-73.986039,199
7,John Sullivan's,40.751629,-73.990512,251
8,Barn Joo 35,40.749756,-73.985940,266
9,The Skylark,40.754105,-73.988731,274


In [156]:
for index, row in df_night_life.iterrows():
    
    #1. MARKER without icon
    nightlife = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    
    #2. Icon
          
    icon = Icon (
            color="blue",
            opacity = 0.6,
            prefix = "fa",
            icon="beer",
            icon_color = "black"
        )
     #3. Marker
    new_marker1 = Marker(**nightlife, icon = icon, radius = 500)
    
    #4. Add the Marker
    new_marker1.add_to(map2)

In [157]:
def getNearbyLocation(lat, long, categorie, radius=1000):

    # create the API request URL
    url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&radius={radius}&categories={categorie}&exclude_all_chains=false&sort=DISTANCE&limit=10"

    # make the GET request
    headers = {
    "accept": "application/json",
    "Authorization": token_fsq
    }    
    response = requests.get(url,headers=headers).json()    
    
    return response

In [158]:
#transportation
transportation_response = getNearbyLocation("40.751892","-73.987592","19047,19049,19046")#19047 rail
#19049 taxi
#19046 metro


coords_1 = (40.751892,-73.987592)
coords_2 = (40.752206,-73.989137)
print (geopy.distance.geodesic(coords_1, coords_2).km)

In [159]:
transportation = []
for i in transportation_response["results"]:
    
    name = i["name"]
    latitude = i["geocodes"]["main"]["latitude"]
    longitude = i["geocodes"]["main"]["longitude"]
    distance = i["distance"]
    
    transportation.append({"name":name, "latitude":latitude, "longitude":longitude, "distance_Optimum Energy": distance})
    
    transportation



In [160]:
df_transportation = pd.DataFrame(transportation)
df_transportation

,name,latitude,longitude,distance_Optimum Energy
0,MTA - 34th Street - Herald Square Subway Station,40.749805,-73.988125,219
1,Go Airlink Shuttle,40.754056,-73.986545,256
2,N Y Checker,40.750484,-73.984964,270
3,Gett,40.752310,-73.991281,314
4,New York Kokusai,40.753953,-73.990385,328
5,LIRR Train - New York Penn Station to Ronkonkoma,40.750688,-73.991392,346
6,Uber Taxi Clone,40.749174,-73.985524,348
7,LIRR Tickets & Information,40.750690,-73.991305,351
8,33rd Street Station,40.748873,-73.988657,370
9,LIRR - Lost & Found,40.750048,-73.991105,384


In [161]:
for index, row in df_transportation.iterrows():
    
    #1. MARKER without icon
    transportation = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    
    #2. Icon
          
    icon = Icon (
            color="blue",
            opacity = 0.6,
            prefix = "fa",
            icon="car",
            icon_color = "black"
        )
     #3. Marker
    new_marker2 = Marker(**transportation, icon = icon, radius = 1000)
    
    #4. Add the Marker
    new_marker2.add_to(map2)

In [162]:
preschool_response = getNearbyLocation("40.751892","-73.987592","12056")

In [163]:
preschool = []
for i in preschool_response["results"]:
    
    name = i["name"]
    latitude = i["geocodes"]["main"]["latitude"]
    longitude = i["geocodes"]["main"]["longitude"]
    distance = i["distance"]
    
    preschool.append({"name":name, "latitude":latitude, "longitude":longitude, "distance_Optimum Energy": distance})
    
    preschool

In [164]:
df_preschool = pd.DataFrame(preschool)
df_preschool

,name,latitude,longitude,distance_Optimum Energy
0,Young Child Expo & Conference,40.754061,-73.992355,467
1,Chelsea Day School,40.747135,-73.985094,573
2,Maryel School of New York,40.747929,-73.981675,664
3,Foundation for Child Develop,40.751816,-73.979599,672
4,KinderCare Learning Center,40.750587,-73.979158,724
5,Metrokids Preschool - Times Square,40.758187,-73.991570,775
6,ReadingEggs,40.744967,-73.989464,785
7,Ivy League Early Learning Academy,40.744927,-73.990948,811
8,Webtoast Internet Services,40.756713,-73.980097,827
9,Murray Hill Wee Ones Club,40.747777,-73.979311,848


In [165]:
for index, row in df_preschool.iterrows():
    
    #1. MARKER without icon
    preschool = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    
    #2. Icon
          
    icon = Icon (
            color="blue",
            opacity = 0.6,
            prefix = "fa",
            icon="child",
            icon_color = "black"
        )
     #3. Marker
    new_marker3 = Marker(**preschool, icon = icon, radius = 500)
    
    #4. Add the Marker
    new_marker3.add_to(map2)

In [166]:
vegan_response = getNearbyLocation("40.751892","-73.987592","13377")
vegan_response

{'results': [{'fsq_id': '5a072c7f419a9e21055e7cb6',
   'categories': [{'id': 13065,
     'name': 'Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}},
    {'id': 13377,
     'name': 'Vegan and Vegetarian Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
      'suffix': '.png'}}],
   'chains': [{'id': 'e9b6ff79-62b5-4a0b-9ef6-b9bf15f3b53f',
     'name': 'sweetgreen'}],
   'distance': 77,
   'geocodes': {'main': {'latitude': 40.752533, 'longitude': -73.987009},
    'roof': {'latitude': 40.752533, 'longitude': -73.987009}},
   'link': '/v3/places/5a072c7f419a9e21055e7cb6',
   'location': {'address': '1384 Broadway',
    'census_block': '360610109001000',
    'country': 'US',
    'cross_street': 'Between 37th St and 38th St',
    'dma': 'New York',
    'formatted_address': '1384 Broadway (Between 37th St and 38th St), New York, NY 10018',
    'locality': 'New York',
    'neighbor

In [167]:
vegan_rest = []
for i in vegan_response["results"]:
    
    name = i["name"]
    latitude = i["geocodes"]["main"]["latitude"]
    longitude = i["geocodes"]["main"]["longitude"]
    distance = i["distance"]
    
    vegan_rest.append({"name":name, "latitude":latitude, "longitude":longitude, "distance_Optimum Energy": distance})
    
    vegan_rest

In [168]:
df_vegan_rest = pd.DataFrame(vegan_rest)


In [169]:
for index, row in df_preschool.iterrows():
    
    #1. MARKER without icon
    vegan_rest = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    
    #2. Icon
          
    icon = Icon (
            color="blue",
            opacity = 0.6,
            prefix = "fa",
            icon="child",
            icon_color = "black"
        )
     #3. Marker
    new_marker4 = Marker(**vegan_rest, icon = icon, radius = 500)
    
    #4. Add the Marker
    new_marker4.add_to(map2)

In [170]:
pet_response = getNearbyLocation("40.751892","-73.987592","11134")


In [171]:
pet_service = []
for i in pet_response["results"]:
    
    name = i["name"]
    latitude = i["geocodes"]["main"]["latitude"]
    longitude = i["geocodes"]["main"]["longitude"]
    distance = i["distance"]
    
    pet_service.append({"name":name, "latitude":latitude, "longitude":longitude, "distance_Optimum Energy": distance})
    
    pet_service

In [172]:
df_pet_service = pd.DataFrame(pet_service)


In [173]:
for index, row in df_pet_service.iterrows():
    
    #1. MARKER without icon
    pet_service = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    
    #2. Icon
          
    icon = Icon (
            color="blue",
            opacity = 0.6,
            prefix = "fa",
            icon="paw",
            icon_color = "black"
        )
     #3. Marker
    new_marker5 = Marker(**pet_service, icon = icon, radius = 1000)
    
    #4. Add the Marker
    new_marker5.add_to(map2)

In [174]:
map2

In [ ]:
#up to 50